In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
import scikitplot as skplt
import matplotlib.pyplot as plt
from simpletransformers.language_representation import RepresentationModel
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import KFold

In [2]:
dataset = pd.read_csv("../../Database/dataBaseWithNER.csv")
dataset = dataset.drop(columns=["Unnamed: 0"])
dataset = dataset.dropna()
target = np.array(dataset["target"].array)

dataset

,email,target
0,start date hourahead timee cardinall hou...,0
1,service long desk price structure deal quote ...,0
2,start date cardinall hourahead timee card...,0
3,start date hourahead timee cardinall anc...,0
4,cardinall deliverable revenue management marke...,0
...,...,...
33340,bio matrix scientific group symbo bmxg p...,1
33341,cardinall step away hot naked webcam girl liv...,1
33342,need pill increase performance click seroius ...,1
33343,datee final nom inlet hpl eastrans car...,0


In [3]:
emailsText = []
for email in dataset["email"]:
    emailsText.append(email)

del dataset

In [4]:
print(len(emailsText))

33341


In [5]:
# skplt.metrics.plot_confusion_matrix(
#     [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#     normalize=True,
#     title="Confusion Matrix",
#     cmap="Purples",
#     hide_zeros=True,
#     figsize=(5,5)
# )
#
# plt.xticks()

# Carregando base de dados  pré-processada

# Representação vetorial GPT2

In [6]:
model=RepresentationModel(
    model_type="gpt2",
    model_name="gpt2",
    use_cuda=True,
    #fp16=True
)

vectorialRepresentation = model.encode_sentences(emailsText, combine_strategy="mean")
vectorialRepresentation.shape

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2ForTextRepresentation: ['h.8.ln_2.weight', 'h.11.attn.c_proj.bias', 'h.8.attn.c_proj.bias', 'h.4.attn.bias', 'h.4.mlp.c_fc.bias', 'h.5.mlp.c_proj.weight', 'h.2.ln_2.weight', 'h.0.mlp.c_fc.weight', 'h.7.attn.c_proj.weight', 'h.7.mlp.c_proj.weight', 'h.7.ln_2.bias', 'h.4.ln_2.bias', 'h.2.ln_1.weight', 'h.10.ln_1.bias', 'h.6.attn.c_attn.bias', 'h.1.mlp.c_proj.weight', 'h.7.ln_1.weight', 'h.4.ln_1.weight', 'h.1.attn.c_proj.weight', 'h.4.attn.c_proj.weight', 'h.11.ln_1.bias', 'h.0.attn.c_proj.weight', 'h.2.attn.bias', 'h.2.attn.c_attn.weight', 'h.2.attn.c_attn.bias', 'h.8.attn.c_attn.weight', 'h.2.mlp.c_proj.weight', 'h.8.mlp.c_fc.bias', 'h.2.mlp.c_fc.weight', 'h.6.ln_2.bias', 'h.4.mlp.c_proj.weight', 'h.9.ln_1.bias', 'h.9.mlp.c_proj.weight', 'h.11.attn.bias', 'h.9.attn.c_attn.bias', 'h.0.mlp.c_fc.bias', 'h.1.attn.c_proj.bias', 'h.10.ln_2.weight', 'h.10.attn.c_proj.weight', 'h.10.attn.c_attn.weight', 'h.7.ln_1.

(33341, 768)

In [7]:
gpt2Dataframe = pd.DataFrame(vectorialRepresentation)
gpt2Dataframe

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.754425,0.789269,-1.293806,0.634251,1.938044,-0.569781,0.377824,1.340950,-1.166817,1.011252,...,-0.191962,1.058731,-2.277187,-0.253051,-0.489028,0.859364,1.936644,1.610118,0.025869,-0.955793
1,0.321773,-0.233847,-0.269172,-0.588057,1.768485,-0.280607,1.047430,-0.086234,-0.406580,0.223166,...,-1.130275,1.211152,-1.572865,-0.519615,1.579095,0.691370,1.846132,-0.937626,-0.170233,0.364428
2,0.896560,0.546716,-1.677532,1.030409,1.285654,-0.619201,0.424408,1.089380,-0.736149,1.082911,...,0.223894,0.580283,-2.416588,-0.078011,-0.320236,0.411358,1.458600,0.928713,-0.239540,-0.354970
3,0.792714,0.782668,-1.189025,0.778011,1.444595,-0.733423,0.294439,1.249979,-1.161031,0.905683,...,0.020335,0.639204,-2.571596,-0.137720,-0.178574,0.334022,1.673328,1.474140,-0.191565,-0.190749
4,0.332344,0.362971,-0.405684,0.397749,0.358304,-0.958279,0.946837,1.157139,-0.058189,1.650188,...,-0.706734,1.681758,-1.429077,0.629436,0.213067,1.147849,0.973108,-0.432340,-0.702642,-0.282397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33336,0.511861,0.832514,-1.399648,1.851290,1.984925,-1.036926,0.451013,0.838663,-0.814666,0.708901,...,-0.188347,0.723476,-2.137387,-0.200568,0.172080,-0.472850,1.178394,1.253184,0.021227,-0.309527
33337,0.977407,0.266629,-1.604069,1.096016,1.205035,-0.459866,0.830633,0.287916,-0.410020,0.202285,...,-0.549093,0.824621,-2.167791,1.013140,0.604684,0.401232,1.207484,-0.720511,-0.354162,-0.222643
33338,0.664014,0.424617,-0.739346,1.113884,0.884731,-0.950039,1.202816,0.975913,0.153484,0.932813,...,-0.160917,-0.168504,-2.553533,0.107064,0.665774,-0.164771,1.526442,0.563803,-0.384363,0.060814
33339,0.850908,0.085265,-1.509501,1.247984,1.079362,-0.508961,0.856407,0.817574,-1.140860,0.928794,...,-0.336332,0.475237,-1.727519,-0.014860,-0.100887,-0.182419,1.447629,0.743375,-0.085994,-0.830510


In [8]:
gpt2Data = np.array(gpt2Dataframe)
gpt2Data

array([[ 0.7544254 ,  0.78926885, -1.2938058 , ...,  1.6101176 ,
         0.02586938, -0.95579314],
       [ 0.3217735 , -0.23384738, -0.26917163, ..., -0.93762565,
        -0.17023289,  0.36442804],
       [ 0.8965603 ,  0.54671633, -1.6775317 , ...,  0.9287129 ,
        -0.23953958, -0.35496992],
       ...,
       [ 0.66401446,  0.42461717, -0.73934567, ...,  0.5638028 ,
        -0.38436264,  0.06081423],
       [ 0.8509083 ,  0.08526485, -1.509501  , ...,  0.7433747 ,
        -0.0859936 , -0.83051026],
       [ 0.9869572 ,  0.21356997, -1.1163118 , ..., -1.0268719 ,
        -0.1928907 , -0.37791866]], dtype=float32)

In [9]:
gpt2Data = gpt2Data.reshape((gpt2Data.shape[0], 1, gpt2Data.shape[1]))

print(gpt2Data.shape)

(33341, 1, 768)


# Visualização de dados com TSNE

In [10]:
# model = TSNE(n_components=2, random_state=0)
# array_red = model.fit_transform(gpt2Dataframe)
#
# df_tsne = pd.DataFrame(array_red)
#
# df_tsne['Target'] = target
# print(df_tsne)
# df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]
#
# df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]
#
# plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')
#
# plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')
#
# plt.title('Dados')
# plt.xlabel('x')
# plt.ylabel('y')
#
# plt.show()

# Validação

In [11]:
foldsAccuracy = []
foldLosses = []

In [12]:
kfold = KFold(n_splits=4, shuffle=True)

In [13]:
foldCount = 1
for train, test in kfold.split(gpt2Data, target):
    model = keras.models.Sequential([
        # keras.layers.LSTM(50, activation='tanh',recurrent_activation='sigmoid',input_shape=[1, gpt2Data.shape[2]]),

        ########## Stacked LSTM
        keras.layers.LSTM(784, activation='relu', return_sequences=True, input_shape=[1, gpt2Data.shape[2]]),
        keras.layers.LSTM(250, activation='relu', return_sequences=True),
        keras.layers.LSTM(90, activation='relu', return_sequences=True),
        keras.layers.LSTM(30, activation='relu'),
        keras.layers.Dense(len(set(target)), activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

    print('****************************************************')
    print(f'Iniciando treinamento da fold: {foldCount}.')

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

    history = model.fit(gpt2Data[train], target[train], epochs=200, callbacks=callbacks, validation_split=0.05)

    scores = model.evaluate(gpt2Data[test], target[test], verbose=0)
    print(f'Score fold {foldCount}: {model.metrics_names[0]} de {scores[0]}; {model.metrics_names[1]} de {scores[1]*100}%')

    foldsAccuracy.append(scores[1] * 100)
    foldLosses.append(scores[0])

    foldCount = foldCount + 1

****************************************************
Iniciando treinamento da fold: 1.
Epoch 1/200
743/743 [==============================] - 10s 11ms/step - loss: 0.6931 - accuracy: 0.5021 - val_loss: 0.6932 - val_accuracy: 0.4908 - lr: 0.0100
Epoch 2/200
743/743 [==============================] - 8s 10ms/step - loss: 0.6931 - accuracy: 0.5035 - val_loss: 0.6930 - val_accuracy: 0.5092 - lr: 0.0100
Epoch 3/200
743/743 [==============================] - 7s 10ms/step - loss: 0.6931 - accuracy: 0.5052 - val_loss: 0.6935 - val_accuracy: 0.4908 - lr: 0.0100
Epoch 4/200
743/743 [==============================] - 7s 10ms/step - loss: 0.6930 - accuracy: 0.5056 - val_loss: 0.6931 - val_accuracy: 0.4908 - lr: 0.0100
Epoch 5/200
743/743 [==============================] - 7s 10ms/step - loss: 0.6930 - accuracy: 0.5065 - val_loss: 0.6934 - val_accuracy: 0.4908 - lr: 0.0100
Epoch 6/200
743/743 [==============================] - 8s 11ms/step - loss: 0.6930 - accuracy: 0.5044 - val_loss: 0.6930 - val_

In [14]:
print('****************************************************')
print('Score de cada fold:')
for i in range(0, len(foldsAccuracy)):
    print('****************************************************')
    print(f'--> Fold {i+1}: Loss: {foldLosses[i]} ; Accuracy: {foldsAccuracy[i]}%')

print('****************************************************')
print('Média de accuracy das folds:')
print(f'--> Accuracy: {np.mean(foldsAccuracy)} (+- {np.std(foldsAccuracy)})')
print(f'--> Loss: {np.mean(foldLosses)}')
print('****************************************************')

****************************************************
Score de cada fold:
****************************************************
--> Fold 1: Loss: 0.254878431558609 ; Accuracy: 90.70297479629517%
****************************************************
--> Fold 2: Loss: 0.6930822730064392 ; Accuracy: 58.224356174468994%
****************************************************
--> Fold 3: Loss: 0.6930049657821655 ; Accuracy: 50.64187049865723%
****************************************************
--> Fold 4: Loss: 0.6930965185165405 ; Accuracy: 56.29274249076843%
****************************************************
Média de accuracy das folds:
--> Accuracy: 63.965485990047455 (+- 15.686328456586708)
--> Loss: 0.5835155472159386
****************************************************


In [15]:
#categories = ["Ham", "Spam"]
#
#skplt.metrics.plot_confusion_matrix(
#    [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#    title="Confusion Matrix",
#    cmap="Purples",
#    hide_zeros=True,
#    figsize=(5,5)
#)
#
#plt.xticks()

In [16]:
# skplt.metrics.plot_confusion_matrix(
#     [categories[i] for i in target], [categories[i] for i in predicoes.tolist()],
#     normalize=True,
#     title="Confusion Matrix",
#     cmap="Purples",
#     hide_zeros=True,
#     figsize=(5,5)
# )
#
# plt.xticks()